# 11 - Gazzetta dello Sport

## Qualunque anno

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.gazzetta.it&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['gs_google_all'] = np.nan

for i in range(300):
    if isinstance(df.gs_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'gs_google_all'] = result
        print((i, df.nome[i]), end=' ')

## 2018

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.gazzetta.it&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['gs_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.gs_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'gs_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

### Pulizia dati

In [16]:
def f(x):
    x = x.replace(',', '')
    x = re.findall('(\d+)', x)
    if x:
        return x[0]
df2 = df.gs_google_all
pd.DataFrame(df2.apply(f)).to_csv('columns/gazzetta_google_all.csv', index=False)

In [34]:
def f(x):
    x = x.replace('.', '')
    x = re.findall('(\d+)', x)
    if x:
        return x[0]
df2 = df.gs_google_18
pd.DataFrame(df2.apply(f)).to_csv('columns/gazzetta_google_18.csv', index=False)